# get_image.ipynb

This Jupyter notebook connects to a Rasbberry Pi 3 Model B+ which is fitted with a 16MP autofocus camera. 

The following steps are implemented:

* An SSH connection is established between this computer and the RPi.
* A command is sent to the RPi which causes it to take a photo: **libcamera-still -o test.png --autofocus**
* FTP is used to copy the image file from the RPi to **OUTPUT_DIR** on this computer.
* Optionally the image file is renamed using the TRAP_ID provided by a QRcode attchd tothe trap.

## Using papermill

For an image containing a QRcode:
```
papermill get_image.ipynb temp.ipynb -p OUTPUT_DIR ./tinian_cycads 
```

For an image without a QRcode:
```
papermill get_image.ipynb temp.ipynb -p OUTPUT_DIR ./tinian_cycads -p USE_QRCODE False -p TRAP_ID 999
```

#### Reference
https://medium.com/@keagileageek/paramiko-how-to-ssh-and-file-transfers-with-python-75766179de73

In [29]:
import paramiko
import cv2
import os
from pyzbar.pyzbar import decode

In [30]:
RPI_IP = "192.168.1.44"
RPI_USERNAME = "pi"
RPI_PASSWORD = "canada12"
TRAP_ID = 999
USE_QRCODE = True
OUTPUT_DIR = './gpepp_nursery'

In [31]:
# Ensure that OUTPUT_DIR exists
if not os.path.exists(OUTPUT_DIR): 
    os.makedirs(OUTPUT_DIR)
    print(f'Created new directory: {OUTPUT_DIR}')

In [32]:
# Open SSH connection to RPi
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(RPI_IP, username=RPI_USERNAME, password=RPI_PASSWORD)

# Take a photo and save it in test.jpg on the RPi
print('Taking photo.')
command = 'libcamera-still -o test.png --autofocus'
ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(command)
output = ssh_stdout.readlines()
print(output)

# Copy photo from RPi to OUTPUTDIR
output_path = f'{OUTPUT_DIR}/{TRAP_ID}.png'
print('Copying image from RPi to {output_path}.')
ftp_client=ssh.open_sftp()
ftp_client.get('/home/pi/test.png', output_path)
ftp_client.close()

# Optionally rename the photo using the QRcode serial number 
if USE_QRCODE:
    img=cv2.imread(output_path)
    # det=cv2.QRCodeDetector()
    code = decode(img)
    val = int(code[0].data)            
    # val, pts, st_code = det.detectAndDecode(img)
    new_path = f'{OUTPUT_DIR}/{val}.png'
    print(f'Renaming image from {output_path} to {new_path}.')
    os.rename(output_path, new_path)

print('Closing SSH connection to RPi.')
ssh.close()

print('Finished')

Taking photo.
['Preview window unavailable\n', 'Still capture image received\n']
Copying image from RPi to {output_path}.
Renaming image from ./gpepp_nursery/999.png to ./gpepp_nursery/32.png.
Closing SSH connection to RPi.
Finished
